# GluonNLP  소개 

GluonNLP 기반으로 Vocab객체와 임베딩을 손쉽게 다루는 방법을 소개한다.

In [2]:
!pip install --pre --upgrade mxnet

    100% |████████████████████████████████| 12.7MB 126kB/s ta 0:00:01
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install gluonnlp

    100% |████████████████████████████████| 163kB 124kB/s ta 0:00:01
  Running setup.py bdist_wheel for gluonnlp ... done
  Stored in directory: /Users/sonwoncheol/Library/Caches/pip/wheels/44/af/bc/20974a8cf42db70b261dfe1eb8e8bf8e52a76c8b192af4d4cd
Successfully built gluonnlp
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
from mxnet import gluon

import warnings
warnings.simplefilter('ignore')

from mxnet import nd
import gluonnlp as nlp
import re
from konlpy.tag import Mecab
import re
mecab = Mecab()

NameError: name 'Tagger' is not defined

## Vocab 객체 

In [210]:
text = """
사과 배 과일 다이어트 : 다이어트에 성공하기 위해서는 식습관과 라이프 스타일을 대대적으로 바꿔야 한다. 여기에 간단한 상상력을 더하면 그 효과를 훨씬 끌어올릴 수 있다. 
다이어트는 의지와 관리가 핵심이다. 엄마 아빠 
아들 딸 구분하지 말고 하나만 낳아서 잘 기르자.
"""

입력된 문장에 대해서 처리 단위가 되는 토큰으로 분리하고 적절한 전처리를 수행한다. 

In [157]:
def mecab_tokenize(source_str):
    return source_str.split()
    #return mecab.morphs(source_str.strip())



counter = nlp.data.count_tokens(mecab_tokenize(text))

In [158]:
counter

Counter({':': 1,
         '간단한': 1,
         '과일': 1,
         '관리가': 1,
         '구분하지': 1,
         '그': 1,
         '기르자.': 1,
         '끌어올릴': 1,
         '낳아서': 1,
         '다이어트': 1,
         '다이어트는': 1,
         '다이어트에': 1,
         '대대적으로': 1,
         '더하면': 1,
         '딸': 1,
         '라이프': 1,
         '말고': 1,
         '바꿔야': 1,
         '배': 1,
         '사과': 1,
         '상상력을': 1,
         '성공하기': 1,
         '수': 1,
         '스타일을': 1,
         '식습관과': 1,
         '아들': 1,
         '아빠': 1,
         '엄마': 1,
         '여기에': 1,
         '위해서는': 1,
         '의지와': 1,
         '있다.': 1,
         '잘': 1,
         '하나만': 1,
         '한다.': 1,
         '핵심이다.': 1,
         '효과를': 1,
         '훨씬': 1})

counter라는 일종의 사전 객체 `{단어, 빈도}`를 기반으로 Vocab객체를 생성한다. 
Vocab 객체는 기 정의된 토큰 `<unk>, <pad>, <bos>, <eos>`을 보유하고 있으며, 사전에 포함될 최소 빈도수로 의미없는 토큰을 제거할 수 있는 인터페이스를 제공한다(`max_size`, `min_freq`). 

In [159]:
vocab = nlp.Vocab(counter)

`token_to_idx`과, `idx_to_token`와 같이 학습셋을 만들때 반드시 구축해야 되는 객체를 만들어 주며, `vocab`객체만 피클로 저장해 모델과 함께 적재할 수 있다. 

In [130]:
vocab.bos_token, vocab.eos_token, vocab.idx_to_token[:10], vocab.token_to_idx['간단'], vocab[['간단', '을']]

('<bos>',
 '<eos>',
 ['<unk>', '<pad>', '<bos>', '<eos>', ':', '간단한', '과일', '관리가', '그', '끌어올릴'],
 0,
 [0, 0])

### Vocab에 임베딩 연결 

GluonNLP가 없던 시점에는 아래와 같이 복잡한 방법으로 임베딩을 사용했다. 

1. 문장 토큰화 및 전처리 
2. 토큰에 대한 딕셔너리 구성 
3. index to token, token to index 자료구조 생성 
4. 임베딩 다운로드 및 보유하고 있는 토큰에 해당되는 임베딩 엔트리로 임베딩 전처리 


Vocab과 Gluon에서 제공하고 있는 임베딩을 이용해 손쉽게 기 학습된 임베딩을 사용해보자! 

여기서 임베딩은 Gluon에서 제공하는 다양한 기 학습된 임베딩 중 FastText 한글 임베딩을 연결해보겠다. 

FastText는 Facebook에서 개발한 임베딩 방법론이며, OOV(out of vocabulary)이슈를 효과적으로 해결한 방법론이다. 



In the opposite direction, we can grab an idex given a token using `vocab.token_to_idx`.

### Attaching word embeddings

아래와 같이 fasttext와 한글 위키 코퍼스로 학습된 임베딩을 가져온다. 

`fasttext` and the named argument `source='wiki.ko'`.

`nlp.embedding.list_sources()`로 모든 기 학습된 임베딩 리스트 출력 

In [91]:
fasttext_simple = nlp.embedding.create('fasttext', source='wiki.ko')

만일 Gensim으로 따로 학습한 임베딩을 사용하고자 한다면 아래와 같이 vocab과 결합할 수 있다. 

`mdl`은 Gensim으로 학습된 모델이다. 
```
gensim_embedding = nlp.embedding.TokenEmbedding(allow_extend=True)
gensim_embedding[mdl.wv.index2word] = nd.array(mdl[mdl.wv.index2word])
vocab.set_embedding(gensim_embedding)
```

To attach the newly loaded word embeddings `fasttext_simple` to indexed words in `vocab`, we simply call vocab's `set_embedding` method:

In [160]:
vocab.set_embedding(fasttext_simple)

결합된 임베딩 shape과 인덱스 크기 비교

In [120]:
vocab.embedding.idx_to_vec.shape, len(vocab.idx_to_token)

((31, 300), 31)

In [213]:
vocab.embedding['<unk>']


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 300 @cpu(0)>

In [132]:
vocab.embedding['사과'], vocab.embedding['배']

(
 [-8.0940e-02  1.3513e-01 -5.4361e-01 -2.9133e-01 -6.2355e-01  1.6854e-01
   1.6384e-01 -6.8300e-02 -1.6073e-01 -2.2654e-01  1.9696e-01  3.4873e-01
  -2.7258e-02  6.7921e-01 -7.2137e-01  4.3859e-01 -1.0719e-02 -2.3379e-02
   2.7814e-02 -3.3025e-01 -3.3353e-01 -2.9284e-01 -9.1742e-02 -4.9350e-01
  -5.4934e-02  7.7779e-02  3.1965e-01 -1.3066e-01  2.3457e-01  7.1430e-01
  -1.7188e-01 -9.6297e-03  2.0930e-02 -4.2738e-01 -1.0519e-01  3.6542e-01
   1.4909e-02  3.8155e-01  2.9169e-01 -6.7597e-02  5.3827e-02 -5.5055e-01
  -5.6948e-01  8.3939e-02  5.2487e-01 -4.1104e-01 -7.9545e-02  1.7147e-01
   6.7951e-01  7.3355e-02  1.2852e-01  2.7254e-01 -4.5853e-01  1.1028e-01
   3.1504e-01 -3.2422e-01 -1.5859e-01  1.2325e-01 -2.3795e-01 -3.2770e-01
   6.7784e-01  5.5509e-01  3.3643e-02 -4.4354e-01 -1.4814e-01 -2.6108e-01
  -1.4608e-02 -6.2265e-01 -1.3015e-01 -1.2823e-02  2.0762e-01 -4.4773e-01
   1.2261e-01 -5.7290e-03 -1.9332e-01  5.9167e-02  5.5691e-01  1.6890e-01
   2.2230e-01  1.9412e-01  3.1604e-0

코사인 유사도 계산
- -1 ~ 1 사이의 값, 1에 가까울 수록 유사도가 크다. 




In [143]:
from mxnet import nd
def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))

x = nd.array([1,2])
y = nd.array([-1,-2])
print(cos_sim(x,x))
print(cos_sim(x,y))


[1.]
<NDArray 1 @cpu(0)>

[-1.]
<NDArray 1 @cpu(0)>


 조사가 붙은 경우 토큰 벡터 도출 가능 

In [141]:
cos_sim(vocab.embedding['다이어트는'], vocab.embedding['다이어트에'])


[0.8691836]
<NDArray 1 @cpu(0)>

In [136]:
cos_sim(vocab.embedding['배'], vocab.embedding['과일'])


[0.342196]
<NDArray 1 @cpu(0)>

In [137]:
cos_sim(vocab.embedding['사과'], vocab.embedding['과일'])


[0.5872627]
<NDArray 1 @cpu(0)>

In [138]:
cos_sim(vocab.embedding['배'], vocab.embedding['사과'])


[0.3542343]
<NDArray 1 @cpu(0)>

### 단어 유사도 

대량의 토큰 벡터중에서 입력된 토큰과 가장 유사한 top N개의 단어 추출 

In [148]:
def norm_vecs_by_row(x):
    return x / nd.sqrt(nd.sum(x * x, axis=1) + 1E-10).reshape((-1,1))

In [153]:
def get_knn(vocab, k, word):
    word_vec = vocab.embedding[word].reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(vocab.embedding.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_vec)
    indices = nd.topk(dot_prod.reshape((len(vocab), )), k=k+1, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    return vocab.to_tokens(indices[1:])

In [161]:
get_knn(vocab, 3, '다이어트')

['다이어트는', '다이어트에', '라이프']

### Word Analogy

We can also apply pre-trained word embeddings to the word
analogy problem. 

"man : woman :: son : daughter"를 analogy 한다. 

'a : b :: c : d' ==  vec('c') + (vec('b')-vec('a')).



In [202]:
fasttext_simple = nlp.embedding.create('fasttext', source='wiki.en')

Embedding file wiki.en.npz is not found. Downloading from Gluon Repository. This may take some time.


In [203]:
vocab = nlp.Vocab(nlp.data.count_tokens(fasttext_simple.idx_to_token),min_freq=1)

vocab.set_embedding(fasttext_simple)

In [204]:
def get_top_k_by_analogy(vocab, k, word1, word2, word3):
    word_vecs = vocab.embedding[word1, word2, word3]
    word_diff = (word_vecs[1] - word_vecs[0] + word_vecs[2]).reshape((-1, 1))
    vocab_vecs = norm_vecs_by_row(vocab.embedding.idx_to_vec)
    dot_prod = nd.dot(vocab_vecs, word_diff)
    indices = nd.topk(dot_prod.reshape((len(vocab), )), k=k, ret_typ='indices')
    indices = [int(i.asscalar()) for i in indices]
    return vocab.to_tokens(indices)

Complete word analogy 'man : woman :: son :'.

In [206]:
get_top_k_by_analogy(vocab, 1, 'man', 'woman', 'son')

['daughter']

word analogy 'beijing : china :: tokyo : '.

In [207]:
get_top_k_by_analogy(vocab, 1, 'beijing', 'china', 'tokyo')

['japan']

word analogy 'bad : worst :: big : '.

In [208]:
get_top_k_by_analogy(vocab, 1, 'bad', 'worst', 'big')

['big']

word analogy 'do : did :: go :'.

In [209]:
get_top_k_by_analogy(vocab, 1, 'do', 'did', 'go')

['went']